# Pipeline para Procesamiento e Ingeniería de Características para Detección de Fraude en Transacciones de Tarjetas de Crédito

Este notebook implementa un pipeline diseñado para una **institución financiera** que busca construir un sistema de detección de fraudes para identificar transacciones sospechosas con tarjetas de crédito. La solución está optimizada para ejecutarse en **Databricks Community Edition**, aprovechando las capacidades de machine learning (ML) fundamentales mientras se trabaja dentro de las limitaciones de la plataforma.


---

## Propósito General del Notebook
1. **Cargar datos crudos:** Importa datos desde un archivo CSV que contiene información de transacciones de tarjetas de crédito.
2. **Ingeniería de características:** Realiza las siguientes transformaciones clave:
   - **Velocidad de transacciones:** Calcula el número de transacciones realizadas por cliente en una ventana temporal definida.
   - **Velocidad de montos:** Calcula el monto total gastado por cliente en la misma ventana temporal.
   - **Perfilado de categoría de comerciante:** Analiza las transacciones realizadas en cada categoría comercial para identificar patrones de gasto.
   - **Patrones temporales:** Extrae características como la hora del día y el día de la semana para detectar comportamientos o anomalías.
3. **Preparación para modelado:** Genera un conjunto de datos enriquecido y listo para entrenar modelos de aprendizaje automático.
4. **Almacenamiento en formato Delta:** Guarda los datos procesados en formato **Delta**, optimizado para consultas y análisis en herramientas como Databricks.

---

## Rutas Necesarias para la Configuración

* **Nota:** Las dependecias necesarias para ejecutarlo están el la primera celda 

### **Archivos de Entrada**
- Los datos originales deben subirse en formato CSV (ejemplo: `credit_card_transactions.csv`).
- Sube estos archivos a la ruta predeterminada: `/FileStore/tables/` en Databricks.
- Ejemplo:

/FileStore/tables/credit_card_transactions.csv


### **Archivos de Salida**
- Los resultados procesados del pipeline se almacenarán en las siguientes rutas:
- **Conjunto de entrenamiento:** `/FileStore/tables/credit_card_transactions_dirt_to_train.csv`.
- **Conjunto de inferencia:** `/FileStore/tables/credit_card_transactions_dirt_to_inf.csv`.
- **Datos enriquecidos en formato Delta:**
  ```python
  output_path = "/FileStore/tables/output_delta_table_datapipe_feature_eng_to_train"
  save_to_delta(df, output_path)
  ```
  Este archivo contiene los datos enriquecidos listos para entrenamiento y su almacenamiento en formato Delta asegura compatibilidad y eficiencia para producción.

---

## Ejecución Completa del Pipeline

### 1. **Cargar Datos Originales**
 - Los datos crudos se cargan desde el archivo CSV ubicado en `/FileStore/tables/credit_card_transactions.csv`.

### 2. **Dividir los Datos en Conjuntos**
 - **80%** para entrenamiento.
 - **20%** para inferencia.  
 - Estos conjuntos se guardan como CSV en las rutas mencionadas.

### 3. **Aplicar Ingeniería de Características**
 - **`transaction_velocity`:** Calcula la velocidad de transacciones.
 - **`amount_velocity`:** Calcula la velocidad de montos.
 - **`merchant_category_profiling`:** Perfilado de categorías comerciales.
 - **`time_based_patterns`:** Extrae patrones temporales.

### 4. **Guardar Datos Procesados**
 - Los datos enriquecidos se guardan en formato Delta para modelado:
   ```
   /FileStore/tables/output_delta_table_datapipe_feature_eng_to_train
   ```

---

## Contenido del Notebook

### **1. Documentación Detallada**
 - Explicaciones de las transformaciones y métodos aplicados.
 - Análisis exploratorio inicial de los datos.

### **2. Experimentos Relevantes**
 - Incluye código y pasos intermedios utilizados durante el desarrollo del pipeline, como pruebas de funciones y validaciones.

### **3. Código Listo para Producción**
 - Todas las transformaciones están encapsuladas en funciones reutilizables.
 - El código final puede trasladarse fácilmente a un **script** o **pipeline de producción** donde sea necesario.

---

## Pasos para Producción

1. **Mover el Código a un Script de Producción**
 - Extrae las funciones y lógica principales del notebook para integrarlas en un script Python independiente o en un sistema ETL automatizado.

2. **Configurar Rutas en el Entorno de Producción**
 - Asegúrate de que las rutas de entrada y salida estén configuradas correctamente para el entorno donde se ejecutará.

3. **Ejecutar en el Entorno Final**
 - El pipeline está diseñado para trabajar en Databricks, pero puede adaptarse para ejecutarse localmente o en otro entorno Spark.

---

## Notas Adicionales
Este notebook combina documentación, experimentos y un pipeline final listo para su implementación en un entorno de producción o investigación avanzada.



In [0]:
!pip install imbalanced-learn
!pip install mlflow
!pip install pytest pytest-cov
!pip install pytest-ipynb 

#%fs ls /FileStore/tables/
#dbutils.fs.rm("/FileStore/tables/engineered_features.delta", True) 

     |████████████████████████████████| 258 kB 6.9 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-591961a2-f31a-465a-8a4f-746b45a920f9/bin/python -m pip install --upgrade pip' command.
  Using cached mlflow-2.19.0-py3-none-any.whl (27.4 MB)
  Using cached SQLAlchemy-2.0.37-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached graphene-3.4.3-py2.py3-none-any.whl (114 kB)
  Using cached docker-7.1.0-py3-none-any.whl (147 kB)
  Using cached flask-3.1.0-py3-none-any.whl (102 kB)
  Using cached mlflow_skinny-2.19.0-py3-none-any.whl (5.9 MB)
  Using cached gunicorn-23.0.0-py3-none-any.whl (85 kB)
  Using cached Markdown-3.7-py3-none-any.whl (106 kB)
  Using cached alembic-1.14.1-py3-none-any.whl (233 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl (64 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl (118 kB)
  Using cached cloudpickle-3.1.1-py3-n

# Initial sampling generation for training and inference 

* Training (80%): 800,000 records.
  * Model building and tuning.
* Inference (20%): 200,000 records.
  * Data to test the model in production.

In [0]:
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.appName("SplitDataset").getOrCreate()

# Ruta del archivo original
file_path = "/FileStore/tables/credit_card_transactions.csv"

# Cargar el DataFrame original
df_muestreo = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(file_path)

# Dividir el DataFrame en 80% para entrenamiento y 20% para inferencia
train_df, inf_df = df_muestreo.randomSplit([0.8, 0.2], seed=42)

# Guardar el DataFrame de entrenamiento
train_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"
train_df.write.format("csv").option("header", "true").mode("overwrite").save(train_path)

# Guardar el DataFrame de inferencia
inf_path = "/FileStore/tables/credit_card_transactions_dirt_to_inf.csv"
inf_df.write.format("csv").option("header", "true").mode("overwrite").save(inf_path)

# Confirmación
print(f"Archivos guardados: \n- Entrenamiento: {train_path} \n- Inferencia: {inf_path}")

Archivos guardados: 
- Entrenamiento: /FileStore/tables/credit_card_transactions_dirt_to_train.csv 
- Inferencia: /FileStore/tables/credit_card_transactions_dirt_to_inf.csv


# Data Pipeline and Feature Engineering

* ### Load and Process the Provided Credit Card Transaction Dataset 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, sum, unix_timestamp, hour, dayofweek
from pyspark.sql.window import Window
import logging

def setup_logging():
    """Setup logging configuration."""
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
    )
    return logging.getLogger(__name__)

logger = setup_logging()

def create_spark_session(app_name="CreditCardFraudDetection"):
    """Create and return a Spark session."""
    try:
        spark = SparkSession.builder \
            .appName(app_name) \
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
            .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
            .getOrCreate()
        logger.info("Spark session created successfully.")
        return spark
    except Exception as e:
        logger.error("Failed to create Spark session: %s", str(e))
        raise

def load_data(spark, file_path):
    """Load data from the provided file path."""
    try:
        df = spark.read.format("csv") \
            .option("header", "true") \
            .option("inferSchema", "true") \
            .load(file_path)
        logger.info("Data loaded successfully from %s.", file_path)
        return df
    except Exception as e:
        logger.error("Failed to load data: %s", str(e))
        raise

def preprocess_data(df):
    """Initial preprocessing: Cast columns to appropriate data types."""
    try:
        df = df.withColumn("amount", col("amount").cast("double"))
        df = df.withColumn("timestamp", col("timestamp").cast("timestamp"))
        logger.info("Data preprocessing completed.")
        return df
    except Exception as e:
        logger.error("Error during preprocessing: %s", str(e))
        raise


In [0]:
## Solo para ver la salida de la ejecución

if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Ruta del archivo
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"  
        
        # Cargar datos desde el archivo
        df = load_data(spark, file_path)

        # Mostrar el DataFrame original
        print("DataFrame Original:")
        df.show(20, truncate=False)

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise


DataFrame Original:
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+
|0000315e14a4  |CUST_00014034|2024-12-11 07:00:53.114|200.84984563271877|retail           |US              |true        |false   |
|00009fe47aa4  |CUST_00019680|2024-12-11 14:54:16.02 |124.88247268225474|entertainment    |US              |false       |false   |
|0001407358f4  |CUST_00020433|2024-12-04 00:46:08.626|156.33773977603684|retail           |US              |false       |false   |
|00018085560f  |CUST_00028904|2024-12-17 17:36:11.758|80.92788170341274 |entertainment    |US              |true        |false   |
|000345fa005c  |CUST_00032284|2024-12-07 00:55:44.622|128.62884

In [0]:
df.schema

Out[5]: StructType([StructField('transaction_id', StringType(), True), StructField('customer_id', StringType(), True), StructField('timestamp', TimestampType(), True), StructField('amount', DoubleType(), True), StructField('merchant_category', StringType(), True), StructField('merchant_country', StringType(), True), StructField('card_present', BooleanType(), True), StructField('is_fraud', BooleanType(), True)])

In [0]:
df.describe().show()

+-------+--------------+-------------+------------------+-----------------+----------------+
|summary|transaction_id|  customer_id|            amount|merchant_category|merchant_country|
+-------+--------------+-------------+------------------+-----------------+----------------+
|  count|        800218|       800218|            800218|           800218|          800218|
|   mean|      Infinity|         null|  123.257890831841|             null|            null|
| stddev|           NaN|         null|110.76529633203918|             null|            null|
|    min|  00000a530069|CUST_00000000|1.0036584716915384|    digital_goods|              CA|
|    max|  ffff74bc4db6|CUST_00051008|  3999.99144862542|           travel|              ZZ|
+-------+--------------+-------------+------------------+-----------------+----------------+



### Análisis de Columnas del DataFrame Original

#### 1. `transaction_id`
- **Conteo:** 800,218 registros.
- **Mínimo y Máximo:** IDs únicos desde `"00000a530069"` hasta `"fffff28ca038"`.
- **Conclusión:** No hay valores faltantes en esta columna, y parece ser un identificador único para cada transacción.

#### 2. `customer_id`
- **Conteo:** 800,218 registros.
- **Rango de IDs:** Desde `"CUST_00000000"` hasta `"CUST_00051010"`.
- **Conclusión:** Todos los registros tienen un cliente asociado, lo que permite agrupar transacciones por cliente para análisis detallado.

#### 3. `amount`
- **Media:** 123.13 (monto promedio de las transacciones).
- **Desviación Estándar:** 109.63 (indica una variabilidad moderada en los montos).
- **Mínimo y Máximo:** Transacciones desde 1.00 hasta 3,999.99.
- **Conclusión:** Hay una gran dispersión en los montos de transacciones, lo que puede reflejar distintos tipos de compras, desde pequeños gastos hasta compras más grandes.

#### 4. `merchant_category`
- **Conteo:** 800,218 registros.
- **Valores Ejemplo:** Incluye categorías como `"digital_goods"`, `"travel"`.
- **Conclusión:** Todas las transacciones están asociadas a una categoría comercial, lo que permite analizar el comportamiento de gasto por categoría.

#### 5. `merchant_country`
- **Rango de Países:** Desde `"CA"` hasta `"ZZ"`.
- **Conclusión:** La columna incluye transacciones internacionales, y `"ZZ"` podría indicar un valor anómalo o de datos no clasificados.


## Transaction Velocity

### **Propósito:**
Calcula el número de transacciones realizadas por cada cliente en los últimos 7 días desde el momento de cada transacción.

### **Proceso:**
1. Convierte la columna `timestamp` a segundos (`timestamp_seconds`) para facilitar cálculos temporales.
2. Define una ventana temporal de 7 días agrupada por `customer_id` y ordenada por tiempo.
3. Calcula la cantidad de transacciones en esa ventana y agrega una nueva columna llamada `transaction_velocity`.

### **Output:**
Un DataFrame que incluye:
- **`timestamp_seconds`**: La fecha de cada transacción en segundos desde Unix Epoch.
- **`transaction_velocity`**: El número de transacciones realizadas por cliente en los últimos 7 días.


In [0]:
def transaction_velocity(df):
    """Calculate transaction velocity (number of transactions per time window)."""
    try:
        df = df.withColumn("timestamp_seconds", unix_timestamp(col("timestamp")))
        time_window = Window.partitionBy("customer_id").orderBy("timestamp_seconds").rangeBetween(-604800, 0)
        df = df.withColumn("transaction_velocity", count("transaction_id").over(time_window))
        logger.info("Transaction velocity calculated.")
        return df
    except Exception as e:
        logger.error("Error calculating transaction velocity: %s", str(e))
        raise

In [0]:
if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Ruta del archivo
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"  
        
        # Cargar datos desde el archivo
        df = load_data(spark, file_path)

        # Aplicar la función transaction_velocity
        df_with_velocity = transaction_velocity(df)

        # Mostrar el DataFrame con la velocidad de transacciones
        print("DataFrame con Velocidad de Transacciones:")
        df_with_velocity.show(20, truncate=False)

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise

DataFrame con Velocidad de Transacciones:
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|timestamp_seconds|transaction_velocity|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+
|f3520fbdab25  |CUST_00000001|2024-07-26 03:50:59.693|27.853296584608096|restaurant       |US              |true        |false   |1721965859       |1                   |
|2c08d3d33433  |CUST_00000001|2024-09-07 08:41:37.913|214.7937768362995 |retail           |US              |false       |false   |1725698497       |1                   |
|acecb0a89e73  |CUST_00000001|2024-09-07 18:30:41.955|160.71036248803097|restaurant       |US              |


## Amount Velocity

### **Propósito:**
Calcula el monto total de las transacciones realizadas por cada cliente en los últimos 7 días desde el momento de cada transacción.

### **Proceso:**
1. Define una ventana temporal de 7 días agrupada por `customer_id` y ordenada por `timestamp_seconds`.
2. Suma los valores de la columna `amount` dentro de la ventana para cada cliente.
3. Agrega una nueva columna llamada `amount_velocity` que contiene el monto total calculado.

### **Output:**
Un DataFrame que incluye:
- **`amount_velocity`**: El monto total de las transacciones realizadas por cliente en los últimos 7 días.


In [0]:
def amount_velocity(df):
    """Calculate amount velocity (total amount per time window)."""
    try:
        time_window = Window.partitionBy("customer_id").orderBy("timestamp_seconds").rangeBetween(-604800, 0)
        df = df.withColumn("amount_velocity", sum("amount").over(time_window))
        logger.info("Amount velocity calculated.")
        return df
    except Exception as e:
        logger.error("Error calculating amount velocity: %s", str(e))
        raise


In [0]:
if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Ruta del archivo
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"
        
        # Cargar datos desde el archivo
        df = load_data(spark, file_path)

        # Aplicar la función transaction_velocity
        df_with_velocity = transaction_velocity(df)

        # Mostrar el DataFrame con la velocidad de transacciones
        print("DataFrame con Velocidad de Transacciones:")
        df_with_velocity.show(20, truncate=False)

        # Aplicar la función amount_velocity sobre df_with_velocity
        df_with_amount_velocity = amount_velocity(df_with_velocity)

        # Mostrar el DataFrame con la velocidad de montos
        print("DataFrame con Velocidad de Montos (Amount Velocity):")
        df_with_amount_velocity.show(20, truncate=False)

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise


DataFrame con Velocidad de Transacciones:
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|timestamp_seconds|transaction_velocity|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+
|f3520fbdab25  |CUST_00000001|2024-07-26 03:50:59.693|27.853296584608096|restaurant       |US              |true        |false   |1721965859       |1                   |
|2c08d3d33433  |CUST_00000001|2024-09-07 08:41:37.913|214.7937768362995 |retail           |US              |false       |false   |1725698497       |1                   |
|acecb0a89e73  |CUST_00000001|2024-09-07 18:30:41.955|160.71036248803097|restaurant       |US              |

## Merchant Category Profiling

### **Propósito:**
Calcula el número de transacciones realizadas por cada cliente en una categoría comercial específica.

### **Proceso:**
1. Define una ventana agrupada por `customer_id` y `merchant_category`.
2. Cuenta el número de transacciones (`transaction_id`) dentro de cada categoría para cada cliente.
3. Agrega una nueva columna llamada `merchant_category_count` que contiene el número total de transacciones por categoría comercial.

### **Output:**
Un DataFrame que incluye:
- **`merchant_category_count`**: El número de transacciones realizadas por cliente en cada categoría comercial.


In [0]:
def merchant_category_profiling(df):
    """Calculate merchant category profiling for each customer."""
    try:
        category_window = Window.partitionBy("customer_id", "merchant_category")
        df = df.withColumn("merchant_category_count", count("transaction_id").over(category_window))
        logger.info("Merchant category profiling completed.")
        return df
    except Exception as e:
        logger.error("Error during merchant category profiling: %s", str(e))
        raise

In [0]:
if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Ruta del archivo
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"
        
        # Cargar datos desde el archivo
        df = load_data(spark, file_path)

        # Aplicar la función transaction_velocity
        df_with_velocity = transaction_velocity(df)

        # Aplicar la función amount_velocity sobre df_with_velocity
        df_with_amount_velocity = amount_velocity(df_with_velocity)

        # Aplicar la función merchant_category_profiling
        df_with_category_profiling = merchant_category_profiling(df_with_amount_velocity)

        # Mostrar el DataFrame con el perfil de categorías comerciales
        print("DataFrame con Perfil de Categorías Comerciales (Merchant Category Profiling):")
        df_with_category_profiling.show(20, truncate=False)

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise


DataFrame con Perfil de Categorías Comerciales (Merchant Category Profiling):
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|timestamp_seconds|transaction_velocity|amount_velocity   |merchant_category_count|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+
|17286088af61  |CUST_00000001|2024-09-21 17:06:23.596|37.07460298352177 |entertainment    |US              |true        |false   |1726938383       |3                   |351.3893987739623 |5                      |
|cac80a102495  |CUST_00000001|2024-10-31 13:20:01.202|199.902522677971

## Time-Based Patterns

### **Propósito:**
Extrae patrones temporales de las transacciones, como la hora del día y el día de la semana, para análisis de comportamiento.

### **Proceso:**
1. Calcula la hora del día a partir de la columna `timestamp` y la agrega como una nueva columna llamada `hour_of_day`.
2. Determina el día de la semana a partir de la columna `timestamp` y lo agrega como una nueva columna llamada `day_of_week`.

### **Output:**
Un DataFrame que incluye:
- **`hour_of_day`**: La hora del día en que se realizó la transacción.
- **`day_of_week`**: El día de la semana en que ocurrió la transacción.


In [0]:
def time_based_patterns(df):
    """Extract time-based patterns like hour of day and day of week."""
    try:
        df = df.withColumn("hour_of_day", hour(col("timestamp")))
        df = df.withColumn("day_of_week", dayofweek(col("timestamp")))
        logger.info("Time-based patterns extracted.")
        return df
    except Exception as e:
        logger.error("Error extracting time-based patterns: %s", str(e))
        raise

In [0]:
if __name__ == "__main__":
    try:
        # Configuración inicial
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Ruta del archivo
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"
        
        # Cargar datos desde el archivo
        df = load_data(spark, file_path)

        # Aplicar la función transaction_velocity
        df_with_velocity = transaction_velocity(df)

        # Aplicar la función amount_velocity sobre df_with_velocity
        df_with_amount_velocity = amount_velocity(df_with_velocity)

        # Aplicar la función merchant_category_profiling
        df_with_category_profiling = merchant_category_profiling(df_with_amount_velocity)

        # Aplicar la función time_based_patterns
        df_with_time_patterns = time_based_patterns(df_with_category_profiling)

        # Mostrar el DataFrame con patrones basados en tiempo
        print("DataFrame con Patrones Basados en Tiempo (Time-Based Patterns):")
        df_with_time_patterns.show(20, truncate=False)

    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise

DataFrame con Patrones Basados en Tiempo (Time-Based Patterns):
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+-----------+-----------+
|transaction_id|customer_id  |timestamp              |amount            |merchant_category|merchant_country|card_present|is_fraud|timestamp_seconds|transaction_velocity|amount_velocity   |merchant_category_count|hour_of_day|day_of_week|
+--------------+-------------+-----------------------+------------------+-----------------+----------------+------------+--------+-----------------+--------------------+------------------+-----------------------+-----------+-----------+
|17286088af61  |CUST_00000001|2024-09-21 17:06:23.596|37.07460298352177 |entertainment    |US              |true        |false   |1726938383       |3                   |351.3893987739623 |5                      |17         |7

### Implement Feature Engineering Using Spark SQL and Window Functions 

In [0]:
def full_feature_engineering(df):
    """Run all feature engineering steps sequentially."""
    try:
        df = transaction_velocity(df)
        df = amount_velocity(df)
        df = merchant_category_profiling(df)
        df = time_based_patterns(df)
        logger.info("All feature engineering steps completed.")
        return df
    except Exception as e:
        logger.error("Error during full feature engineering: %s", str(e))
        raise

### Store Engineered Features in Delta Format

In [0]:
def save_to_delta(df, output_path):
    """Save processed data to Delta format."""
    try:
        if not output_path:
            raise ValueError("Output path cannot be empty.")

        df.write.format("delta").mode("overwrite").save(output_path)
        logger.info("Data saved to Delta format at %s.", output_path)
    except Exception as e:
        logger.error("Failed to save data to Delta: %s", str(e))
        raise

### Production-Ready Code
* Main Function for Pipeline Execution

In [0]:
if __name__ == "__main__":
    try:
        # Setup
        logger.info("Starting the data pipeline.")
        spark = create_spark_session()

        # Load data
        file_path = "/FileStore/tables/credit_card_transactions_dirt_to_train.csv"
        df = load_data(spark, file_path)

        # Preprocess data
        df = preprocess_data(df)

        # Perform feature engineering
        df = full_feature_engineering(df)

        # Save the processed data
        output_path = "/FileStore/tables/output_delta_table_datapipe_feature_eng_to_train"
        save_to_delta(df, output_path)

        logger.info("Data pipeline completed successfully.")
    except Exception as e:
        logger.error("Pipeline execution failed: %s", str(e))
        raise

# Pruebas Unitarias para Funciones de Procesamiento

## 1. `test_preprocess_data`
### **Propósito:**
Verificar que `preprocess_data`:
- Convierte `amount` a `double` y `timestamp` a `timestamp`.
- Conserva las columnas necesarias.

### **Validación:**
La prueba pasa si las columnas `amount` y `timestamp` están presentes en el DataFrame procesado.

---

## 2. `test_transaction_velocity`
### **Propósito:**
Confirmar que `transaction_velocity`:
- Calcula correctamente la velocidad de transacciones en una ventana de 7 días.
- Agrega la columna `transaction_velocity`.

### **Validación:**
La prueba pasa si el DataFrame resultante contiene la columna `transaction_velocity`.

**Nota:** Estas pruebas unitarias, debido a limitaciones de tiempo y a la imposibilidad de ejecutarlas directamente en el mismo notebook, se dejaron de forma hipotética (no las ejecuté como tal, pero aún asi las hice por el requerimiento). Según la metodología de pytest, estas pruebas se ejecutan en archivos `.py`. Por esta razón, decidí priorizar otros pasos del proceso.


In [0]:
import pytest
from pyspark.sql import SparkSession

# Crear una sesión de Spark para pruebas
@pytest.fixture(scope="session")
def spark():
    return SparkSession.builder \
        .appName("PytestSparkSession") \
        .master("local[*]") \
        .getOrCreate()

# Crear un DataFrame de prueba
@pytest.fixture
def sample_data(spark):
    data = [
        ("T1", "C1", "2025-01-01 10:00:00", 100.0, "retail", "US", True),
        ("T2", "C1", "2025-01-02 11:00:00", 150.0, "retail", "US", False),
    ]
    schema = ["transaction_id", "customer_id", "timestamp", "amount", "merchant_category", "merchant_country", "card_present"]
    return spark.createDataFrame(data, schema)

# Definir las pruebas
def test_preprocess_data(sample_data):
    df = preprocess_data(sample_data)
    assert "amount" in df.columns
    assert "timestamp" in df.columns

def test_transaction_velocity(sample_data):
    df = preprocess_data(sample_data)
    df = transaction_velocity(df)
    assert "transaction_velocity" in df.columns

In [0]:
#!pytest --maxfail=5 --disable-warnings


In [0]:
#!pytest --cov=. --cov-report=term-missing
